## Preparing the workspace for CNTK in jupyter notebook

In [5]:
#r "netstandard"
#load "Paket.fsx"

* Unless you reference *netstandard* this way anything CNTK related will be marked as errors and will be ignored by ignore by autocomplete.

* *Paket.fsx* is neccessary to enable IfSharp's paket extensions such as <span style='color:#B21543'>Paket.Dependencies.Install</span>. Downloading paket.exe for each notebook specifically is thus unnessecary.

### 1. Install neccessary packages

In [6]:
Paket.Dependencies.Install """
framework: netstandard2.0
generate_load_scripts: true
storage: none
source https://nuget.org/api/v2
nuget CNTK.CPUOnly
nuget MathNet.Numerics
nuget MathNet.Numerics.FSharp
"""

Usually in F# notebooks we tend to see <span style="color:green">paket.dependencies</span> created this way:

> Paket.Package ["CNTK.CPUOnly"; "FsLab"; "FSharp.Charting"]

I used <span style='color:#B21543'>Paket.Dependencies.Install</span> instead of <span style='color:#B21543'>Paket.Package</span> as it allows us to set **storage: none** and save some disk space and bandwidth by not re-downloading the same dependencies every time a project needs CNTK support, some of the required DLLs being rather immense as it is.

This is slightly ironic, considering that in order to use CNTK in jupyter we end up having to copy all dependencies in a common bin path anyway.

### 2. Reference Cntk.Core.Managed-#.#.dll from the nuget packages folder

In [5]:
#load @".paket\load\main.group.fsx"

System.Reflection.Assembly.GetAssembly(typeof<CNTK.CNTKLib>).CodeBase
|> printfn "CNTK managed binary referenced from:\n\t%s"

CNTK managed binary referenced from:
	file:///C:/Users/cernu/.nuget/packages/cntk.cpuonly/2.6.0/lib/netstandard2.0/Cntk.Core.Managed-2.6.dll


I prefer the reference location to be something like **file://&lt;user folder&gt;/packages/**. If you always end up with a reference in the IfSharp.exe path it is because it seems to have precedence as far as the <span style="color:green">#load</span> directive is concerned.

This also means that once *main.group.fsx* has been generated in the IfSharp folder (for instance after downloading the visualization tools) you will no longer have access to the local generated scripts until you delete those in the IfSharp folder.

If you get an error the likes of:
>The namespace or module 'CNTK' is not defined.

after <span style='color:#B21543'>Paket.Dependencies.Install</span> above succeeds, it is exceedingly likely that you need to clean the IfSharp folder from paket generated files.

If you ommit **storage: none** from <span style='color:#B21543'>Paket.Dependencies.Install</span>, the IfSharp folder is also where any nuget packages will end up.

### 3. Copy all CNTK dependencies to the same folder

Currently all CNTK dlls need to be copied in the same folder with _Cntk.Core.Managed-#.#.dll_ else they can't be resolved at runtime. The following script finds the path of the presently referenced Cntk DLL and uses it to locate any dependencies. Should work for both GPU and CPUOnly.

Visual Studio does this by default in some projects, but here we need to do it by hand:

In [6]:
#load "fsx/prepareWorkspace.fsx"

let localBinFolder = "bin"

PrepareWorkspace.createOrCleanLocalBinFolder localBinFolder
PrepareWorkspace.copyDependenciesToLocalFolder localBinFolder (PrepareWorkspace.Release) 

Copying from 'packages\cntk.cpuonly\2.6.0\lib\netstandard2.0\Cntk.Core.Managed-2.6.dll'
Copying from 'packages\cntk.cpuonly\2.6.0\support\x64\Release\Cntk.Composite-2.6.dll'
Copying from 'packages\cntk.cpuonly\2.6.0\support\x64\Release\Cntk.Core-2.6.dll'
Copying from 'packages\cntk.cpuonly\2.6.0\support\x64\Release\Cntk.Core.CSBinding-2.6.dll'
Copying from 'packages\cntk.cpuonly\2.6.0\support\x64\Release\Cntk.Deserializers.Binary-2.6.dll'
Copying from 'packages\cntk.cpuonly\2.6.0\support\x64\Release\Cntk.Deserializers.HTK-2.6.dll'
Copying from 'packages\cntk.cpuonly\2.6.0\support\x64\Release\Cntk.Deserializers.Image-2.6.dll'
Copying from 'packages\cntk.cpuonly\2.6.0\support\x64\Release\Cntk.Deserializers.TextFormat-2.6.dll'
Copying from 'packages\cntk.cpuonly\2.6.0\support\x64\Release\Cntk.Math-2.6.dll'
Copying from 'packages\cntk.cpuonly\2.6.0\support\x64\Release\Cntk.PerformanceProfiler-2.6.dll'
Copying from 'packages\cntk.deps.opencv.zip\2.6.0\support\x64\Dependency\Release\opencv_w

So far I haven't been able to reference debug DLLs successfully in a notebook, i.e. without hitting the dreaded <span style='font-family:consolas'>DllNotFoundException: Cntk.Core.CSBinding-#.#.dll</span>, so I suggest you also stick to the release builds.

**Known bug**: If you have *CNTK.gpu* in your packets folder then it's own references will be copied here as well, regardless of whether you've had paket only add *CNTK.CPUOnly*. Other than the waste of space this seems to have no other ramifications.

### 4. Reference CNTK from the newly created folder

<span style="color:blue">If you intend to use CNTK calls in this notebook you should restart the kernel after copying is complete, and then reference _Cntk.Core.Managed-#.#.dll_ from the newly created _/bin_ folder.</span>

In [3]:
#r @"bin\Cntk.Core.Managed-2.6.dll"
#load @".paket\load\main.group.fsx"

open CNTK
let cpu = DeviceDescriptor.UseDefaultDevice()
printfn "Congratulations, you are using CNTK for: %A" (cpu.Type)

System.Reflection.Assembly.GetAssembly(typeof<CNTK.CNTKLib>).CodeBase
|> printfn "\nCNTK managed binary referenced from:\n\t%s"

Congratulations, you are using CNTK for: CPU

CNTK managed binary referenced from:
	file:///C:/Users/cernu/Documents/Code/IfCntk/notebooks/cntk-tutorials/bin/Cntk.Core.Managed-2.6.dll


### 5. Update path variable

Finally, we need to update the path variable to inlcude the newly created /bin folder, otherwise we get an error when instantiating the learner before training.
>CNTK.CNTKLibPINVOKE.SGDLearner__SWIG_1 exception

<span style="color: blue">This is only affects the current process, so you'll have to do this again whenever you run a cntk notebook.</span>

In [4]:
let path = System.Environment.GetEnvironmentVariable("PATH")
let path' = sprintf "%s%c%s" path (System.IO.Path.PathSeparator) (System.IO.Path.GetFullPath("bin"))
System.Environment.SetEnvironmentVariable("PATH", path')

 When calling CNTK from script make sure it's through a 64bit version of fsi.exe, such as fsiAnyCPU.exe, or you will run into _bad image_ exceptions.